In [1]:
import numpy as np
import datajoint as dj
import trimesh
from collections import Counter
pinky = dj.create_virtual_module("pinky","microns_pinky")
schema = dj.schema("microns_pinky")

Connecting celiib@10.28.0.34:3306


In [12]:
pinky.SpineValidationStatistics()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,author_original name of last editor,author_proofreader name of last editor,starting_index starting index for the slice generation algorithm,n_triangles number of triangles used for slice,date_time the last time it was edited,"n_auto_spine number of faces classified as spine,spine-head or spine-neck by auto-classifier",n_auto_shaft number of faces classified as shaft by auto-classifier,"n_false_negatives number of spines incorrectly autoclassified as shaft (merged spines would add 2,3,4...etc)",n_false_positive number of sections of the shaft that were incorrectly classified as spine,true_positives number of spines correctly classified as spines,n_merged_spines total number of additional spines that resulted from being merged with others,perc_spines_with_merges percentage of the true positive count that also had other spines merged onto it,n_spines_with_overspined total number out of the true positives that had overspining occuring on their labeling,"average_percentage_overspined for those that had overspining, the average face percentage that were labeled as overspined",avg_spine_size average number of faces for the true positive spines
3,648518346341371119,0.35,Oblique,0,computer_Auto,js48,17292,20142,2019-08-22 13:13:55,7572,12570,0,2,43,5,0.10256,2,0.20838,187.97436
3,648518346349471562,0.35,Apical,0,computer_Auto,js48,108090,30000,2019-08-12 17:19:47,14534,15466,3,0,75,11,0.10000,0,None,205.70000
3,648518346349471910,0.35,Basal,2,computer_Auto,js48,31952,30000,2019-08-16 12:46:29,16408,13592,8,1,84,23,0.19737,0,None,213.28947
3,648518346349473044,0.35,Apical,0,computer_Auto,js48,7895,30000,2019-08-12 18:41:01,16234,13766,0,0,90,14,0.15789,3,0.55761,197.94737
3,648518346349473160,0.35,Basal,1,computer_Auto,js48,119413,30000,2019-08-12 17:22:11,14215,15785,10,1,88,17,0.17284,0,None,171.67901
3,648518346349473804,0.35,Apical,0,computer_Auto,js48,78047,30000,2019-08-27 16:33:10,4236,25764,1,1,28,0,0.00000,0,None,162.84000
3,648518346349473813,0.35,Basal,2,computer_Auto,js48,102954,30000,2019-08-12 17:23:35,12450,17550,6,0,60,13,0.16364,0,None,224.70909
3,648518346349473821,0.35,Apical,0,computer_Auto,js48,144601,30000,2019-08-12 17:25:45,18618,11382,1,0,87,16,0.15663,1,0.19072,221.65060
3,648518346349473835,0.35,Apical,0,computer_Auto,js48,18050,30000,2019-08-12 17:26:29,16059,13941,2,0,84,20,0.19753,0,None,196.66667
3,648518346349473838,0.35,Basal,3,computer_Auto,js48,11184,20963,2019-08-14 12:33:13,7307,13656,1,0,39,1,0.02857,1,0.73622,192.97143


In [5]:
@schema
class SpineValidationStatisticsOrphan(dj.Computed):
    definition="""
    -> pinky.ProofreadLabelSpineWithInterestSlicesOrphan
    ---
    n_auto_spine      : int unsigned                 # number of faces classified as spine,spine-head or spine-neck by auto-classifier
    n_auto_shaft      : int unsigned                 # number of faces classified as shaft by auto-classifier
    n_false_negatives :  int unsigned                # number of spines incorrectly autoclassified as shaft (merged spines would add 2,3,4...etc)
    n_false_positive  : int unsigned                 # number of sections of the shaft that were incorrectly classified as spine
    true_positives    : int unsigned                 # number of spines correctly classified as spines
    n_merged_spines   : int unsigned                 # total number of additional spines that resulted from being merged with others
    perc_spines_with_merges=null : decimal(6,5)           # percentage of the true positive count that also had other spines merged onto it
    n_spines_with_overspined: int unsigned           # total number out of the true positives that had overspining occuring on their labeling
    average_percentage_overspined=null: decimal(6,5)      # for those that had overspining, the average face percentage that were labeled as overspined
    avg_spine_size=null    : decimal(10,5)                # average number of faces for the true positive spines
    
    """
    
    key_source = pinky.ProofreadLabelSpineWithInterestSlicesOrphan() & dict(author_proofreader="js48",status="complete")
    
    def make(self,key):
        print("key  = " + str(key))
        significant_size = 50
        search_key = dict(author_proofreader="js48",status="complete")
        search_dict = dict(search_key,segmentation=key["segmentation"],
                          segment_id=key["segment_id"],
                          compartment_type=key["compartment_type"],
                          component_index=key["component_index"])
        
        # get the mesh and the original and proofread labels
        """
        way to get the entire mesh:
        1) get the id, compartment type and component index, starting index, number of faces, and triangle indices
        from the validation proofing table
        2) Get the whole mesh from: 
        a. pinky.PymeshfixDecimatedExcitatoryStitchedMesh
        b. pinky.Decimation35OrphanStitched

        3) Use the triangle indices to restrict to the actual portion of submesh

        """

        #Example
        verts,faces = (pinky.Decimation35OrphanStitched() & search_dict).fetch1("vertices","triangles")

        #get compartment verts and faces
        verts_comp,faces_comp = (pinky.CompartmentOrphan.ComponentOrphan & search_dict).fetch1("vertex_indices","triangle_indices")

        #build the trimesh object
        new_mesh = trimesh.Trimesh(verts,faces)
        #new_mesh.show()
        compartment_mesh = new_mesh.submesh([faces_comp],append=True)

        #now get the submesh according to the triangle indices
        tri_indices = (pinky.ProofreadLabelSpineWithInterestSlicesOrphan & search_dict).fetch1("triangle_indices")
        random_mesh = compartment_mesh.submesh([tri_indices],append=True)
        #random_mesh.show()

        # how to get the automatic labels: 
        verts_labels,triangles_labels = (pinky.ComponentLabelOrphanWithStats & search_dict).fetch1("labeled_vertices","labeled_triangles")

        verts_labels = verts_labels.astype("int")
        triangles_labels = triangles_labels.astype("int")


        #get the new triangle lables based on the submesh indexes
        auto_labels = triangles_labels[tri_indices]
        auto_labels[auto_labels == 0] = 17
        auto_labels[auto_labels == 12] = 14
        auto_labels[auto_labels == 13] = 14
        auto_labels[auto_labels == 15] = 14
        auto_labels = auto_labels
        auto_labels

        #how to get the proofread labels
        tri_labels_proofread = (pinky.ProofreadLabelSpineWithInterestSlicesOrphan & search_dict).fetch1("triangles")
        tri_labels_proofread
        
        #get rid of all of the error and no_label sections: 
        labels_dict = dict(
        spine_1=12,
            shaft=2,
            shaft_over_spined=18,
            shaft_3=5,
            remove_label=0,
            error=10,
            spine_2=3,
            spine_3=4,
            spine_4=6,
            spine_5=9,
            all_one_spine=11,
            auto_shaft=17,
            auto_spine=14,
            spine_head_manual=13,
            spine_neck_manual=15,
            label_removed=1
        )

        labels_dict_flipped = dict([(v,k) for k,v in labels_dict.items()])
        labels_dict_flipped
        
        """
        1) Handle the all-one-spine pieces: get the submesh and split pieces, count the number of pieces as false negatives (SIGNIFICANT SIZED) 
         Eliminate any mesh pieces that are error or no label
        """
        all_one_spine_n_faces = 0
        all_one_spine_n_bodies = 0

        x = np.where(tri_labels_proofread == labels_dict["all_one_spine"])[0]
        if len(x) > 0:
            all_one_spine_n_faces = len(x)
            #get the submesh
            all_one_spine_mesh = random_mesh.submesh([x],append=True)
            #get the number of splits
            all_one_spine_mesh_split = all_one_spine_mesh.split(only_watertight=False)
            all_one_spine_n_bodies = len(all_one_spine_mesh_split)

        print("all_one_spine_n_bodies = " + str(all_one_spine_n_bodies)) 
        print("all_one_spine_n_faces = " + str(all_one_spine_n_faces))

        print("Done finding all in one spines")

        
        """
        For Shaft: 
        1) From original mesh extract the indices that were auto-shaft (get the number of faces in submesh to store later)
        2) Use the shaft indices to extract the new proofread labels for the autoshaft

        --- set size threshold for the spine counting (50)
        4) *** DON'T HAVE TO DO ANYMORE **** Get all the all_one_spine label indices --> create submesh from those. Count the number of all_one_spine as a missed spine
        5) Get the submesh of anything that is spine_1,spine_2,spine_3,spine_4,spine_5
        6) Split the submesh into disjointed parts (using connected_components to get the indices)
        7) Loop through the list (with indices)
        - if not meet desired threshold then put as 0
        - count the number of different spine labels that exist
        - store in array
        8) Sum the array and add on the all-one-spine number to count the total number missed
        """

        #extract the indices that were auto-shaft

        auto_shaft_indices = np.where(auto_labels==labels_dict["auto_shaft"])[0]
        #random_mesh_auto_shaft_indices = clean_indices[auto_shaft_indices]
        n_auto_shaft = len(auto_shaft_indices)
        n_auto_shaft

        #extract teh proofread shaft labels
        tri_labels_proofread_shaft = tri_labels_proofread[auto_shaft_indices]
        total_Counter = Counter(tri_labels_proofread_shaft)
        shaft_counts = dict([(labels_dict_flipped[k],total_Counter[k]) for k in dict(total_Counter).keys() ])
        print("After extracting the shaft after cleaning Distribution is:" + str(shaft_counts) )


        #get the indexes of all of the spines
        spine_numbers = [labels_dict["spine_" + str(k)] for k in range(1,6)] 
        print(spine_numbers)
        # np.where(tri_labels_proofread_clean_shaft 
        spine_indexes = np.array([])

        for iy in spine_numbers: 
            print("Working on spine "+ str(labels_dict_flipped[iy]))
            spine_indexes = np.hstack((spine_indexes, np.where(tri_labels_proofread_shaft==iy)[0]))
            print(f"Len of spine indexes = {len(spine_indexes)}")
        spine_indexes = spine_indexes.astype("int")
        #check to see if there were any spines: 
        total_spine_count_list = []

        if (spine_indexes.any()):
            #get the spines only submesh
            spine_submesh = random_mesh.submesh([auto_shaft_indices[spine_indexes]],append=True)
            #divide the spine submesh into disconnected portions
            spine_disconnected_groups = trimesh.graph.connected_components(edges=spine_submesh.face_adjacency,
                                          nodes=np.arange(len(spine_submesh.faces)),
                                          min_len=1,
                                          engine=None)
            #REMINDER: these indices correspond to the spine mesh vertices (which are spine indexes of auto_shaft of random mesh)
            print(len(spine_disconnected_groups))

            if len(spine_disconnected_groups)>0:
                for i,spine_group in enumerate(spine_disconnected_groups):
                    if len(spine_group) < significant_size:
                        print("Size of group too small for a spine")
                        continue
                    print("Working on shaft spine #" + str(i))
                    #iterate through and see what spine labels are present
                    original_labels_for_spine = tri_labels_proofread_shaft[spine_indexes[spine_group]]
                    print(Counter(original_labels_for_spine))

                    #see how many spine labels are in the proofread version labels for this spine
                    current_spine_count = 0
                    for iy in spine_numbers: 

                        #print("Working on spine "+ str(labels_dict_flipped[iy]))
                        if len(np.where(original_labels_for_spine==iy)[0]) > 0: 
                            print(f"Found {iy} in spine group")
                            current_spine_count += 1

                    total_spine_count_list.append(current_spine_count)

        print(total_spine_count_list)
        total_false_negative = sum(total_spine_count_list)
        print("total_false_negative = " + str(total_false_negative))
        
        """
        For spines: 
        1) From original mesh extract indices that were auto-spine (get the number of faces in submesh)
        2) Use auto-spine indices to extract the proofread labels for autoshaft
        --- set size threshold
        4) Dividee the submesh into all disjointed pieces (count the number of pieces and save off)
        5) Calculate the number of shaft and over-spine and percentage there are (WILL CALCULATE THIS AS ITERATING THROUGH)
        5) Count the number of pieces that are 70% or greater as false positives and remove those indices from the list (FALSE POSITIVES)
        6) For each piece still left in the list
        - count the number of different spine labels that exist (if only auto-spine then tally that as perfect)
        - store in array
        6) Sum up the array and tally up the number that got perfect
        """

        #extract the indices that were auto-shaft

        auto_spine_indices = np.where(auto_labels==labels_dict["auto_spine"])[0]
        #random_mesh_auto_shaft_indices = clean_indices[auto_shaft_indices]
        n_auto_spine = len(auto_spine_indices)
        print("n_auto_spine = " + str(n_auto_spine))

        #extract teh proofread spine labels
        tri_labels_proofread_spine = tri_labels_proofread[auto_spine_indices]
        total_Counter = Counter(tri_labels_proofread_spine)
        spine_counts = dict([(labels_dict_flipped[k],total_Counter[k]) for k in dict(total_Counter).keys() ])
        print("After extracting the shaft after cleaning Distribution is:" + str(spine_counts) )


        #get the indexes of all of the spines
        spine_numbers = [labels_dict["spine_" + str(k)] for k in range(1,6)]
        spine_numbers_and_spine_overspine = spine_numbers + [labels_dict["auto_spine"],labels_dict["shaft_over_spined"],
                                                             labels_dict["shaft"],
                                                             labels_dict["shaft_3"]
                                                            ]
        #print(spine_numbers_and_spine_overspine)
        # np.where(tri_labels_proofread_clean_shaft 
        spine_indexes = np.array([])

        for iy in spine_numbers_and_spine_overspine: 
            #print("Working on spine "+ str(labels_dict_flipped[iy]))
            spine_indexes = np.hstack((spine_indexes, np.where(tri_labels_proofread_spine==iy)[0]))
            print(f"Len of spine indexes = {len(spine_indexes)}")
        spine_indexes = spine_indexes.astype("int")
        #check to see if there were any spines: 
        additional_spine_count_list = []
        overspined_list = []
        overspined_list_perc = []
        spine_size_list = []
        false_positive_count = 0
        auto_distinct_spines = 0

        #
        if (spine_indexes.any()):
            #get the spines only submesh
            spine_submesh = random_mesh.submesh([auto_spine_indices[spine_indexes]],append=True)
            #divide the spine submesh into disconnected portions
            spine_disconnected_groups = trimesh.graph.connected_components(edges=spine_submesh.face_adjacency,
                                          nodes=np.arange(len(spine_submesh.faces)),
                                          min_len=1,
                                          engine=None)
            #REMINDER: these indices correspond to the spine mesh vertices (which are spine indexes of auto_shaft of random mesh)
            #print(len(spine_disconnected_groups))


            if len(spine_disconnected_groups)>0:
                auto_distinct_spines = len(spine_disconnected_groups)
                for i,spine_group in enumerate(spine_disconnected_groups):

                    if len(spine_group) < significant_size:
                        print("Size of group too small for a spine")
                        continue




                    #print("Working on shaft spine #" + str(i))
                    #iterate through and see what spine labels are present
                    original_labels_for_spine = tri_labels_proofread_spine[spine_indexes[spine_group]]
                    original_labels_counter = Counter(original_labels_for_spine)
                    #print(original_labels_counter)

                    #get the number of shaft faces
                    current_shaft_counter = 0
                    if labels_dict["shaft_3"] in original_labels_counter.keys():
                        current_shaft_counter += original_labels_counter[labels_dict["shaft_3"]]
                    if labels_dict["shaft"] in original_labels_counter.keys():
                        current_shaft_counter += original_labels_counter[labels_dict["shaft"]]

                    #count the amount it was labeled as a shaft
                    if current_shaft_counter/len(spine_group) > 0.70:
                        print("Found false positive for spine " + str(i))
                        false_positive_count += 1
                        continue

                    spine_size_list.append(len(spine_group))


                    if labels_dict["shaft_over_spined"] in original_labels_counter.keys(): 
                        overspined_list.append(original_labels_counter[labels_dict["shaft_over_spined"]])
                        overspined_list_perc.append(original_labels_counter[labels_dict["shaft_over_spined"]]/len(spine_group))
                    else:
                        overspined_list.append(0)
                        overspined_list_perc.append(0)

                    #see how many spine labels are in the proofread version labels for this spine
                    current_spine_count = 0
                    for iy in spine_numbers: 

                        #print("Working on spine "+ str(labels_dict_flipped[iy]))
                        if len(np.where(original_labels_for_spine==iy)[0]) > 0: 
                            #print(f"Found {iy} in spine group")
                            current_spine_count += 1

                    additional_spine_count_list.append(current_spine_count)

#         print("additional_spine_count_list = " + str(additional_spine_count_list))

#         print("overspined_list = " + str(overspined_list))

#         print("overspined_list_perc = " + str(overspined_list_perc))

#         print("false_positive_count = " + str(false_positive_count))

#         print("auto_distinct_spines = " + str(auto_distinct_spines))

#         print("spine_size_list = " + str(spine_size_list))
                                                               
        #the final stats that wanted
        #print("additional_spine_count_list = " + str(additional_spine_count_list))

        #stats want to keep from the shaft

        #stats we want to keep from the spine

        total_false_negative = sum(total_spine_count_list)
        print("total_false_negative = " + str(total_false_negative))

        n_merged_spines = sum(additional_spine_count_list)
        print("n_merged_spines = " + str(n_merged_spines))
        perc_spines_with_merges = len(np.where(np.array(additional_spine_count_list)>0)[0])/len(additional_spine_count_list)
        print("perc_spines_with_merges = " + str(perc_spines_with_merges))

        #print("overspined_list = " + str(overspined_list))
        overspined_faces = np.where(np.array(overspined_list)>0)[0]
        n_spines_with_overspined = len(overspined_faces)
        print("n_spines_with_overspined = " + str(n_spines_with_overspined))

        #print("overspined_list_perc = " + str(overspined_list_perc))
        #average percentage of overspined 
        average_percentage_overspined = np.mean(np.array(overspined_list_perc)[overspined_faces])
        print("average_percentage_overspined = " + str(average_percentage_overspined))

        print("false_positive_count = " + str(false_positive_count))

        print("auto_distinct_spines = " + str(auto_distinct_spines))
        avg_spine_size = np.mean(spine_size_list)
        print("avg_spine_size = " + str(avg_spine_size))

        print('n_auto_spine = ' + str(n_auto_spine))
        print('n_auto_shaft = ' + str(n_auto_shaft))

        """
        n_auto_spine      : int unsigned                 # number of faces classified as spine,spine-head or spine-neck by auto-classifier
        n_auto_shaft      : int unsigned                 # number of faces classified as shaft by auto-classifier
        n_false_negatives :  int unsigned                # number of spines incorrectly autoclassified as shaft (merged spines would add 2,3,4...etc)
        n_false_positive  : int unsigned                 # number of sections of the shaft that were incorrectly classified as spine
        true_positives    : int unsigned                 # number of spines correctly classified as spines
        n_merged_spines   : int unsigned                 # total number of additional spines that resulted from being merged with others
        perc_spines_with_merges : decimal(6,5)           # percentage of the true positive count that also had other spines merged onto it
        n_spines_with_overspined: int unsigned           # total number out of the true positives that had overspining occuring on their labeling
        average_percentage_overspined: decimal(6,5)      # for those that had overspining, the average face percentage that were labeled as overspined
        avg_spine_size    : decimal(10,5)                # average number of faces for the true positive spines
        """
        
        
        #insert into the table
        new_dict = dict(key,
                       n_auto_spine=n_auto_spine,
                       n_auto_shaft=n_auto_shaft,
                       n_false_negatives=total_false_negative,
                       n_false_positive=false_positive_count,
                       true_positives=auto_distinct_spines,
                       n_merged_spines=n_merged_spines,
                       perc_spines_with_merges=perc_spines_with_merges,
                       n_spines_with_overspined=n_spines_with_overspined,
                       average_percentage_overspined=average_percentage_overspined,
                       avg_spine_size=avg_spine_size)

        self.insert1(new_dict,skip_duplicates=True)


In [ ]:
#(schema.jobs & "table_name='__spine_validation_statistics_orphan'").delete()

In [ ]:
# key  = {'segmentation': 3, 'segment_id': 648518346341371119, 'decimation_ratio': 0.35, 'compartment_type': 'Oblique', 'component_index': 0}

In [6]:
import time
start_time = time.time()
SpineValidationStatisticsOrphan.populate(reserve_jobs=True)
print(f"Total time = {time.time() - start_time}")

key  = {'segmentation': 3, 'segment_id': 648518346341357855, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Dendrite', 'component_index': 0, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 58577, 'n_triangles': 30000, 'date_time': datetime.datetime(2019, 8, 27, 12, 52, 4)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 12689, 'spine_1': 392}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 392
Working on spine spine_2
Len of spine indexes = 392
Working on spine spine_3
Len of spine indexes = 392
Working on spine spine_4
Len of spine indexes = 392
Working on spine spine_5
Len of spine indexes = 392
1
Working on shaft spine #0
Counter({12: 392})
Found 12 in spine group
[1]
total_false_negative = 1
n_auto_spine = 16919
After extracting the shaft after cleaning Distribution is:{'auto_spine': 15103, 'spine_1': 1114, 'spi

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


key  = {'segmentation': 3, 'segment_id': 648518346341361018, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Dendrite', 'component_index': 0, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 65851, 'n_triangles': 30000, 'date_time': datetime.datetime(2019, 8, 20, 13, 8, 18)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 15876}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 0
Working on spine spine_2
Len of spine indexes = 0
Working on spine spine_3
Len of spine indexes = 0
Working on spine spine_4
Len of spine indexes = 0
Working on spine spine_5
Len of spine indexes = 0
[]
total_false_negative = 0
n_auto_spine = 14124
After extracting the shaft after cleaning Distribution is:{'auto_spine': 13590, 'spine_2': 87, 'spine_1': 447}
Len of spine indexes = 447
Len of spine indexes = 534
Len of spine indexes = 534
Len of 

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


key  = {'segmentation': 3, 'segment_id': 648518346341366161, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Dendrite', 'component_index': 0, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 3257, 'n_triangles': 30000, 'date_time': datetime.datetime(2019, 8, 14, 11, 9, 9)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 18060, 'shaft': 1}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 0
Working on spine spine_2
Len of spine indexes = 0
Working on spine spine_3
Len of spine indexes = 0
Working on spine spine_4
Len of spine indexes = 0
Working on spine spine_5
Len of spine indexes = 0
[]
total_false_negative = 0
n_auto_spine = 11939
After extracting the shaft after cleaning Distribution is:{'auto_spine': 10937, 'shaft': 133, 'spine_1': 509, 'spine_2': 145, 'spine_3': 215}
Len of spine indexes = 509
Len of spine indexes

...

key  = {'segmentation': 3, 'segment_id': 648518346349496702, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Dendrite', 'component_index': 0, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 7792, 'n_triangles': 16252, 'date_time': datetime.datetime(2019, 8, 23, 14, 29, 38)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 5420, 'spine_1': 372}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 372
Working on spine spine_2
Len of spine indexes = 372
Working on spine spine_3
Len of spine indexes = 372
Working on spine spine_4
Len of spine indexes = 372
Working on spine spine_5
Len of spine indexes = 372
2
Working on shaft spine #0
Counter({12: 209})
Found 12 in spine group
Working on shaft spine #1
Counter({12: 163})
Found 12 in spine group
[1, 1]
total_false_negative = 2
n_auto_spine = 8675
After extracting the shaft afte

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


key  = {'segmentation': 3, 'segment_id': 648518346349509521, 'decimation_ratio': Decimal('0.35'), 'compartment_type': 'Dendrite', 'component_index': 0, 'author_original': 'computer_Auto', 'author_proofreader': 'js48', 'starting_index': 46277, 'n_triangles': 30000, 'date_time': datetime.datetime(2019, 8, 22, 16, 43, 21)}
all_one_spine_n_bodies = 0
all_one_spine_n_faces = 0
Done finding all in one spines
After extracting the shaft after cleaning Distribution is:{'auto_shaft': 13538, 'error': 721}
[12, 3, 4, 6, 9]
Working on spine spine_1
Len of spine indexes = 0
Working on spine spine_2
Len of spine indexes = 0
Working on spine spine_3
Len of spine indexes = 0
Working on spine spine_4
Len of spine indexes = 0
Working on spine spine_5
Len of spine indexes = 0
[]
total_false_negative = 0
n_auto_spine = 15741
After extracting the shaft after cleaning Distribution is:{'auto_spine': 14356, 'error': 198, 'spine_1': 1021, 'spine_2': 166}
Len of spine indexes = 1021
Len of spine indexes = 1187
L

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Total time = 134.58749556541443


In [ ]:
fc = dj.create_virtual_module("pinky_fc","microns_pinky_fc")

In [ ]:
fc.ContactCorrelation.describe()